# Welcome to Semantic Kernel with Microsoft Fabric - Using the Planner
- This notebook has following examples:
    - install semantic -kernel library in Microsoft Fabric Spark environment, if required
    - use the BasicPlanner
    - import skills from the LakeHouse as well as use inline
- Todo: Secrets are hardcoded here - to move to Azure Keyvault


In [1]:
%pip show semantic-kernel

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 5, Finished, Available)

Name: semantic-kernel
Version: 0.3.8.dev0
Summary: 
Home-page: 
Author: Microsoft
Author-email: SK-Support@microsoft.com
License: 
Location: /home/trusted-service-user/cluster-env/trident_env/lib/python3.10/site-packages
Requires: aiofiles, numpy, openai, openapi_core, prance, pydantic, python-dotenv, regex
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
#By adding library in workspace setting, you have already installed semantic-kernel! 
#in case you have not setup in workspace setting, then uncomment below line and run this cell
#
#%pip install semantic-kernel --quiet

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 6, Finished, Available)

In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureTextCompletion,
    AzureChatCompletion
)

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 7, Finished, Available)

In [4]:
text_completion_deployment = "text-davinci-003"
chat_completion_deployment = "gpt-35-turbo" 
api_key = "*****"
endpoint = "https://*****.openai.azure.com/"
#print(text_completion_deployment, chat_completion_deployment, api_key, endpoint)

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 8, Finished, Available)

In [5]:
kernel = sk.Kernel()
kernel.add_text_completion_service("text_completion", AzureTextCompletion(text_completion_deployment, endpoint, api_key))



StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 9, Finished, Available)

In [6]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 10, Finished, Available)

In [7]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "/lakehouse/default/Files/sk-sample-plugins"
imported_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
imported_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
imported_skill = kernel.import_skill(TextSkill(), "TextSkill")

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 11, Finished, Available)

In [8]:
ask = """
Suggest day trip ideas for Labor Day Long weekend in Seattle metro area. Rewrite in better English.
"""
plan = await planner.create_plan_async(ask, kernel)
print(plan.generated_plan)

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 12, Finished, Available)

    {
        "input": "Labor Day Long Weekend in Seattle Metro Area",
        "subtasks": [
            {"function": "WriterSkill.Brainstorm"},
            {"function": "WriterSkill.EnglishImprover"}
        ]
    }


In [9]:
response = await planner.execute_plan_async(plan, kernel)
print(response)

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 13, Finished, Available)

1. Pay a visit to the Seattle Aquarium
2. Take a guided tour of the renowned Space Needle
3. Investigate the Pike Place Market
4. Tour the Museum of Pop Culture
5. Take a leisurely walk around the Olympic Sculpture Park
6. Tour the Woodland Park Zoo
7. Spend a day at Seattle Waterfront
8. Take a ferry ride to Bainbridge Island
9. Explore the Chihuly Garden and Glass
10. Enjoy a picnic at Discovery Park


In [10]:
ask = """
Suggest day trip ideas for Labor Day Long weekend in Seattle metro area. Write in French.
"""
plan = await planner.create_plan_async(ask, kernel)
print(plan.generated_plan)

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 14, Finished, Available)

    {
        "input": "Labor Day Long Weekend Day Trip Ideas in Seattle Metro Area",
        "subtasks": [
            {"function": "WriterSkill.Brainstorm"},
            {"function": "WriterSkill.Translate", "args": {"language": "French"}}
        ]
    }


In [11]:
response = await planner.execute_plan_async(plan, kernel)
print(response)

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 15, Finished, Available)

1. Visitez le musée d'art de Seattle
2. Regardez un match des Mariners au T-Mobile Park
3. Prenez un ferry pour Bainbridge Island
4. Visitez la Tour de l'Espace
5. Passez la journée à la plage de Golden Gardens
6. Faites une randonnée à Snoqualmie Falls
7. Explorez le front de mer de Seattle
8. Visitez les écluses Hiram M. Chittenden
9. Faites du shopping au Pike Place Market
10. Faites une visite du Seattle Underground


In [12]:
ask = """
Suggest day trip ideas for Labor Day Long weekend in Seattle metro area.
Write as a poem.
"""

#Assume Poem skill not available in the Lakehouse file/directory, 
#notice that we didn't import previously when we imported other skills to begin with.
#following inline function and skill can be created
sk_prompt = """
{{$input}}

Rewrite the above as a romantic poem.
"""
inlineSkillFunction = kernel.create_semantic_function(sk_prompt, "GenPoem", "GenPoemSkill",
                                                      max_tokens=1000, temperature=1.2)
inlineSkillFunction = kernel.create_semantic_function(sk_prompt, "Poem", "GenPoemSkill",
                                                      max_tokens=1000, temperature=1.2)


plan2 = await planner.create_plan_async(ask, kernel)
print(plan2.generated_plan)

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 16, Finished, Available)

    {
        "input": "Labor Day Long weekend in Seattle metro area",
        "subtasks": [
            {"function": "WriterSkill.Brainstorm"},
            {"function": "GenPoemSkill.Poem"}
        ]
    }


In [13]:
response2 = await planner.execute_plan_async(plan2, kernel)
print(response2)

StatementMeta(, f51bfe6a-49e3-4344-9d24-b0c461123a19, 17, Finished, Available)


Tired of the day,
Away lets us stray,
To the Seattle Aquarium,
where love will sway.

And then on a ferry,
We shall to escape,
Enchanted island,
We visit to make.

Picnic at the park,
Adventure begins there,
Laughter, smiles and romance,
How can we not share? 

Hands held as we walk,
The Seattle Arboretum we roam,
Clicking pics through each stroll,
Will create marvelous memories to own.

The Underground Tour was quite a sight,
Time shouldn’t we fear,
For moments like these here,
Will live longer, staying near. 

A flight of fancy is what we must take,
At The Museum of Flight we will partake,
Get the window seat, delicious things I’ll partake,
A journey through time, both together we’ll make. 

The Space Needle our conquest,
Let’s explore to no agreement,
The stars shining bright in this moment,
Gift memories wrap in wondrous sentiment. 

Dinner by the waterside, 
Feast to days pursuance,
Seafood for our fill appetite,
Should match the spirit of romance. 

Exploring through Pike Market

##### End of this Notebook